In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import random

#metti final data >= 2016

data = pd.read_json("finalTrumpPreprocessed.json")
print(len(data))
data = data[data["year"] >= 2016] 
print(len(data))
data.columns


Using TensorFlow backend.


36159
10325


Index(['in_reply_to_user_id_str', 'is_retweet', 'text', 'month', 'year',
       'hour', 'week_year', 'date', 'modded_text', 'modded1', 'noTags_punct',
       'modded1_stemmed'],
      dtype='object')

In [2]:
data = data[data["is_retweet"] == False] 
print(len(data))


10325


In [3]:
#remove links
data["modded1"] = data["text"].str.replace("http\S+"," ")

data["modded1"] = data["modded1"].str.replace("\n", " ")

#remove punctuation
data['modded1'] = data["modded1"].str.replace("[^\w\s]"," ")
data["modded1"] = data["modded1"].str.lower()
data["modded1"]

25834    i will be on  foxnews live   with members of m...
25835                  happy new year  amp  thank you     
25836    happy new year from  maralago  thank you to my...
25837      jallenaip  hillary said she was in a  fog of...
25838      sprinklermanus   cnn  realdonaldtrump they r...
25839    well  the year has officially begun  i have ma...
25840      marie7777777777   realdonaldtrump we love u ...
25841      jodil792  we are standing with you  spreadin...
25842      memeoryhead  i m one of your biggest fans mr...
25843      casuperrunner   georgehenryw huckabee is a g...
25844    i will be going to mississippi tomorrow night ...
25845      codyraymille  i have never been interested i...
25846    the person that hillary clinton least wants to...
25847    thank you so much to   for naming me the 2015 ...
25848     votetrump2016  amp  together we will  makeame...
25849    massive crowds expected in mississippi tomorro...
25850      jebbush is a sad case  a total embarrassment.

In [4]:
sentences = data["modded1"].tolist()
list_train_set = sentences

In [5]:
fullText = " ".join(sentences)
text_in_words = [w for w in fullText.split(' ') if w.strip() != '' or w == '\n']
text_in_words = list(map(lambda s: s.strip(), text_in_words))

print('Corpus length in words:', len(text_in_words))

from collections import Counter
word_freq = dict(Counter(fullText.split()))

not_freq = [x for x in word_freq if word_freq[x] < 19] 

shorter = []
for sentence in sentences:
    t = sentence.split()
    q = sentence
    for el in t:
        if el in not_freq:
            q = q.replace(el," ")
    if q != []:
        shorter.append([q])
        
unique = len(word_freq.keys())-len(not_freq)
print("words", len(word_freq.keys()))
print("not frequent", len(not_freq))
print("remaining", len(word_freq.keys())-len(not_freq))


Corpus length in words: 255726
words 12742
not frequent 11245
remaining 1497


In [6]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

vocab = Counter(fullText.split())
#print(vocab)

corpus = [x.split() for x in sentences]
#corpus

In [7]:
from gensim.models import Word2Vec


w2vec_size = 200
w2 = Word2Vec(corpus, size=w2vec_size, window=5, min_count=1,sg=1)


In [49]:
word_vectors = w2.wv
#print("Number of word vectors: {}".format(len(word_vectors.vocab)))
w2.wv.most_similar(positive=['clinton'])


[('crooked', 0.9407603740692139),
 ('dnc', 0.8912924528121948),
 ('paid', 0.8794649839401245),
 ('clear', 0.8755836486816406),
 ('dirty', 0.8678460121154785),
 ('email', 0.8631898760795593),
 ('frame', 0.8619242310523987),
 ('scheme', 0.8608847856521606),
 ('emails', 0.8606369495391846),
 ('her', 0.8600423336029053)]

In [9]:
MAX_NB_WORDS = len(word_vectors.vocab)
MAX_SEQUENCE_LENGTH = 100

In [10]:
from keras.preprocessing.sequence import pad_sequences
import keras.utils as ku


word_index = {t[0]: i+1 for i,t in enumerate(vocab.most_common(MAX_NB_WORDS))}
#print(word_index)
sequences = [[word_index[t] for t in sent.split()] for sent in sentences[:len(list_train_set)]]
test_sequences = [[word_index[t]  for t in sent.split()] for sent in sentences[len(list_train_set):]]


In [ ]:
word_index

In [11]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1] #i+1
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]


[[9, 10],
 [9, 10, 15],
 [9, 10, 15, 11],
 [9, 10, 15, 11, 168],
 [9, 10, 15, 11, 168, 390],
 [9, 10, 15, 11, 168, 390, 19],
 [9, 10, 15, 11, 168, 390, 19, 596],
 [9, 10, 15, 11, 168, 390, 19, 596, 4],
 [9, 10, 15, 11, 168, 390, 19, 596, 4, 37],
 [9, 10, 15, 11, 168, 390, 19, 596, 4, 37, 424]]

In [12]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = 4
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    #riprova il padding con pre
    predictors, lab = input_sequences[:,:-1],input_sequences[:,-1] #tolto -1
    label = ku.to_categorical(lab, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
input_len = max_sequence_len-1

#data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="pre", truncating="post")
#data
print(predictors.shape)
label.shape

(245404, 3)


(245404, 12743)

In [13]:
print(predictors[0],predictors[1],predictors[2])

[0 0 9] [ 0  9 10] [ 9 10 15]


In [14]:
#WEIGHT MATRIX
WV_DIM = w2vec_size

nb_words = total_words
# we initialize the matrix with random numbers
wv_matrix = (np.random.rand(total_words, WV_DIM) - 0.5) / 5.0
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        # words not found in embedding index will be all-zeros.
        wv_matrix[i] = embedding_vector
    except:
        pass
print(nb_words)

12743


In [15]:
from keras.layers import Dense, Input, Embedding, Dropout,LSTM,Bidirectional
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

In [16]:

model = Sequential()

model.add(Embedding(nb_words, w2vec_size, mask_zero=False,weights=[wv_matrix],input_length=input_len,trainable=True)) #mask zero = False
model.add(Bidirectional(LSTM(800)))
model.add(Dropout(0.4))
model.add(Dense(nb_words, activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

#con 800 meno di 20 epoch! un po' overfitta, 10

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3, 200)            2548600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1600)              6406400   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 12743)             20401543  
Total params: 29,356,543
Trainable params: 29,356,543
Non-trainable params: 0
_________________________________________________________________


In [43]:
#model_json = model.to_json()
#with open("definitiveMaybe.json", "w") as json_file:
#    json_file.write(model_json)
    
model.save_weights('definitiveMaybe15ItersMaybeTooOverfitting.h5')


In [17]:
from keras.callbacks import ModelCheckpoint

filepath="postelectionGenerator-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [35]:
history = model.fit(predictors, label, validation_split=0.35, epochs=2, verbose=1)#callbacks=callbacks_list)
#5 + 5 + 3 + 2

Train on 159512 samples, validate on 85892 samples
Epoch 1/2
159512/159512 [==============================] - 1519s 10ms/step - loss: 1.4213 - acc: 0.6797 - val_loss: 7.3288 - val_acc: 0.1690
Epoch 2/2
159512/159512 [==============================] - 1518s 10ms/step - loss: 1.3754 - acc: 0.6918 - val_loss: 7.4454 - val_acc: 0.1697


In [ ]:
#history = model.fit(predictors, label, validation_split=0.40, epochs=25, verbose=1,callbacks=callbacks_list)
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

model.load_weights("postelectionGenerator-12-0.16.hdf5")


In [ ]:
print(word_index)

In [24]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    preds = []
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0] 
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='post') #max sequence len - 1
        predicted = model.predict_classes(token_list, verbose=0)
        preds.append(predicted)
        output_word = ""
        
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
#    print(preds)
    return seed_text.title()


#load another model
import json
from keras.models import model_from_json

with open('veryGoodAndBigOne.json', 'r') as json_file:
    model = model_from_json(json_file.read())
model.load_weights("veryGoodAndBigOne.h5")

In [ ]:
model.summary()

In [56]:
generate_text("best socks ever", 32, model, 3)


'Best Socks Ever There They Are No Longer A Credible Source Of The Many Wonderful Things That He Stood For Honor Him For Being The Final Two And Heading Into A September Runoff In Alabama'

In [41]:
rand = sentences
sampling = random.choices(rand, k=10)

seeds = []
for el in sampling:
    seeds.append(" ".join(el.split()[:3]))
    
for el in seeds:
    print("seed: \" " + el + "\"")
    t = generate_text(el, 24,model, input_len)
    print(t + "\n")

seed: " the local politicians"
The Local Politicians And Build The Wall Democrats Are Protecting Ms 13 Thugs Being Hit On The No Russian Collusion In The History Of The Fbi And

seed: " i wish people"
I Wish People Are The Real Deal And Those That Want To Make A Deal With Iran 1 In Terror No Problem Is Doing So Badly Compared

seed: " i asked vp"
I Asked Vp Pence To Leave Stadium If Any Players Kneeled Disrespecting Our Country Or Our Flag They Said It Loud And Clear She Spent Big Money

seed: " a democratic lawmaker"
A Democratic Lawmaker Just Introduced A Bill To Repeal Obamacare And Give Americans Many Choices And Much Lower Prices At The Pharmacy Counter Intelligence Operation Into The

seed: " total inaction on"
Total Inaction On Daca By Dems Where Are You A Deal Can Be Made Without Causing Disruption The Focus Must Be Life And Safety Daca Which Is

seed: " sc has kept"
Sc Has Kept Us Safe From Exec Amnesty For Now But They Are Fading Fast And The Dems Remember The Dirty Dossier U

In [42]:
for sentence in sentences:
    s = sentence.split()
    if "vp"  in s and "kneeled" in s and "players" in s:
        print(sentence + "\n")

i asked  vp pence to leave stadium if any players kneeled  disrespecting our country  i am proud of him and  secondlady karen 



In [ ]:
generate_text("Citterio is buonista and his communist", 5 , model, input_len)


In [ ]:
goodSeeds = ["why is that","we are with","donald trump is","hillary clinton is a bad","great success today at","i am hearing so","get out and vote","melania and i offer","we must stop being"]
    
for el in goodSeeds:
    print("seed: \" " + el + "\"")
    t = generate_text(el, 24,model, max_sequence_len)
    print(t + "\n")

In [ ]:
for sentence in sentences:
    s = sentence.split()
    if "network"  in s and "news" in s and "become" in s :
        print(sentence + "\n")